In [ ]:
!pip install torch sentence-transformers pandas PyPDF2 python-docx openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 18.5 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, InputExample, losses

# Optional: for PDFs and DOCX
from PyPDF2 import PdfReader
from docx import Document

# ----------------------------
# Mount Google Drive
# ----------------------------
from google.colab import drive
drive.mount('/content/drive')

# ----------------------------
# File processing (works for paths and file-like objects)
# ----------------------------
def process_file(file):
    """
    Extract text from file.
    Supports: PDF, DOCX, TXT, CSV, XLSX
    Can accept: file path (str) or file-like object
    """
    text = ""

    if isinstance(file, str):
        if file.endswith(".pdf"):
            reader = PdfReader(file)
            for page in reader.pages:
                text += page.extract_text() or ""
        elif file.endswith(".txt"):
            with open(file, "r", encoding="utf-8") as f:
                text = f.read()
        elif file.endswith(".docx"):
            doc = Document(file)
            text = "\n".join([p.text for p in doc.paragraphs])
        elif file.endswith(".xlsx"):
            df = pd.read_excel(file)
            text = df.astype(str).agg(" ".join, axis=1).to_string()
        elif file.endswith(".csv"):
            df = pd.read_csv(file)
            text = df.astype(str).agg(" ".join, axis=1).to_string()
    else:
        if file.name.endswith(".pdf"):
            import tempfile
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(file.read())
                tmp_file_path = tmp_file.name
            reader = PdfReader(tmp_file_path)
            for page in reader.pages:
                text += page.extract_text() or ""
            os.remove(tmp_file_path)
        elif file.name.endswith(".txt"):
            text = file.read().decode("utf-8")
        elif file.name.endswith(".docx"):
            doc = Document(file)
            text = "\n".join([p.text for p in doc.paragraphs])
        elif file.name.endswith(".xlsx"):
            df = pd.read_excel(file)
            text = df.astype(str).agg(" ".join, axis=1).to_string()
        elif file.name.endswith(".csv"):
            df = pd.read_csv(file)
            text = df.astype(str).agg(" ".join, axis=1).to_string()
    return text

# ----------------------------
# Chunking utility
# ----------------------------
def chunk_text(text, chunk_size=500, overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

# ----------------------------
# Load legal datasets
# ----------------------------
def load_legal_training_data(file_paths):
    texts = []
    for path in file_paths:
        if not os.path.exists(path):
            print(f"⚠️ File not found: {path}")
            continue
        try:
            raw_text = process_file(path)
            chunks = chunk_text(raw_text)
            texts.extend(chunks)
        except Exception as e:
            print(f"⚠️ Skipping {path}, error: {e}")
    return texts

# ----------------------------
# Train Legal-BERT
# ----------------------------
def train_model(
    output_path="/content/drive/My Drive/AI CHATBOT/fine_tuned_legal_model",
    epochs=2,
    batch_size=16
):
    """
    Fine-tune Legal-BERT on your datasets
    """

    # Define file paths (update to your Drive)
    file_paths = [
        "/content/drive/My Drive/AI CHATBOT/DATA/ipc_sections.csv",
        "/content/drive/My Drive/AI CHATBOT/DATA/FIR_DATASET.csv",
        "/content/drive/My Drive/AI CHATBOT/DATA/Indian Penal Code Book.pdf",
    ]

    # Load all texts
    texts = load_legal_training_data(file_paths)
    if not texts:
        print("❌ No texts loaded. Check file paths.")
        return

    # Create InputExamples (self-similarity task)
    examples = [InputExample(texts=[txt, txt], label=1.0) for txt in texts]

    # Load pretrained Legal-BERT
    model = SentenceTransformer("nlpaueb/legal-bert-base-uncased")

    # DataLoader and Loss
    train_dataloader = DataLoader(examples, shuffle=True, batch_size=batch_size)
    train_loss = losses.CosineSimilarityLoss(model)

    # Train
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=epochs,
        warmup_steps=100,
        show_progress_bar=True
    )

    # Save fine-tuned model
    os.makedirs(output_path, exist_ok=True)
    model.save(output_path)
    print(f"✅ Fine-tuned model saved at {output_path}")

# ----------------------------
# Run training
# ----------------------------
if __name__ == "__main__":
    train_model()


Mounted at /content/drive


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tushar24681012 (tushar24681012-jain-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.000000
1000,0.000000


✅ Fine-tuned model saved at /content/drive/My Drive/AI CHATBOT/fine_tuned_legal_model
